<a href="https://colab.research.google.com/github/eringgao/lucy_listings/blob/main/lucyslistings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations and Declarations

In [5]:
%pip install pandas
%pip install geopy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.3 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
from geopy import distance
import requests
import json
import datetime
import math
import itertools


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Coding Starts Here

In [ ]:
# Use for Google Colab
df = pd.read_csv("/content/drive/MyDrive/rapid_data.csv")

print(df)

In [8]:
# Use locally
df = pd.read_csv("data/rapid_data.csv")

print(df)

    Unnamed: 0  bathrooms  bedrooms           city country currency  \
0            0        2.0         4     Pittsburgh     USA      USD   
1            1        1.0         2  Mt Washington     USA      USD   
2            2        1.5         3     Pittsburgh     USA      USD   
3            3        2.0         4     Pittsburgh     USA      USD   
4            4        1.0         1     Pittsburgh     USA      USD   
5            5        1.0         3     Pittsburgh     USA      USD   
6            6        1.0         2     Pittsburgh     USA      USD   
7            7        3.0         5     Pittsburgh     USA      USD   
8            8        1.5         2     Pittsburgh     USA      USD   
9            9        2.0         3     Pittsburgh     USA      USD   
10          10        2.0         3     Pittsburgh     USA      USD   
11          11        1.0         2     Pittsburgh     USA      USD   
12          12        1.0         4     Pittsburgh     USA      USD   
13    

In [10]:
'''
  Create an OOP structure for how we want to store all of our data.
  Example format: 

    class Listing: # abstraction?
      def get_type(self) -> str: 
        # Manual, Zillow, CMU
        pass 
      def get_housing_info(self) -> str: 
        # String data (paragraphs?) about housing
        pass
      def get_address(self) -> dict: 
        pass
      def distance_from_cmu(self) -> float: 
        pass

    class SchoolListing(Listing): 
      def __init__(self, address): 
        # Some stuff here
        self.get_type = "CMU"
        # self.distance_from_cmu 
'''

In [11]:
addresses = df.loc[:, "streetAddress"]
lat = df.loc[:, "latitude"]
longi = df.loc[:, "longitude"]
cmu_address = (40.4433, -79.944023)
distance_df = pd.DataFrame(columns = ['Address1', 'Latitude', 'Longitude'])
distance_df['Address1'] = addresses
distance_df['Latitude'] = lat
distance_df['Longitude'] = longi
print(distance_df)
walking = []
driving = []
for i in range(len(addresses)):
  d = distance.distance((distance_df.loc[i, "Latitude"], distance_df.loc[i, "Longitude"]), (cmu_address[0],cmu_address[1]))
  walking.append(d.miles)
  r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{distance_df.loc[i, "Longitude"]},{distance_df.loc[i, "Latitude"]};{cmu_address[1]},{cmu_address[0]}?overview=false""")
  route_1 = json.loads(r.content)["routes"][0]
  driving.append(route_1["distance"]*0.000621371)
distance_df["Walking Distance"] = walking
distance_df["Driving Distance"] = driving
print(distance_df)



                         Address1   Latitude  Longitude
0               2353 Sidney St #2  40.429380 -79.971140
1              441 Edgemont St #2  40.422600 -79.998880
2                   200 S 15th St  40.424866 -79.983440
3                 5317 Forbes Ave  40.440155 -79.934940
4              4711 Maripoe St #1  40.455498 -79.949670
5                   1625 Hybla St  40.474262 -80.037970
6             3202 Hazelhurst Ave  40.383247 -79.965300
7                4245 Parkman Ave  40.448345 -79.957280
8              1007 Galveston Ave  40.453010 -80.017100
9                 11370 Althea Rd  40.468670 -79.832700
10              333 Lehigh Ave #2  40.456150 -79.926100
11  1465 Old Steubenville Pike #B  40.450570 -80.114400
12                223 Carnegie Pl  40.446510 -79.898950
13         5541 Beacon St FLOOR 1  40.434210 -79.928990
14               6116 Stanton Ave  40.468600 -79.918110
15               2801 Larkins Way  40.425310 -79.966675
16                 637 Danbury St  40.468822 -80

In [ ]:
class Listing: 
    def __init__(self):
        

In [18]:
# print(type(distance_df))
print(distance_df["Address1"])

0                 2353 Sidney St #2
1                441 Edgemont St #2
2                     200 S 15th St
3                   5317 Forbes Ave
4                4711 Maripoe St #1
5                     1625 Hybla St
6               3202 Hazelhurst Ave
7                  4245 Parkman Ave
8                1007 Galveston Ave
9                   11370 Althea Rd
10                333 Lehigh Ave #2
11    1465 Old Steubenville Pike #B
12                  223 Carnegie Pl
13           5541 Beacon St FLOOR 1
14                 6116 Stanton Ave
15                 2801 Larkins Way
16                   637 Danbury St
17              1806 Rhine St #1804
18              4008 Woolslayer Way
19              3220 1/2 Hardie Way
20              514 Parallel Ave #2
21              2525 Sarah St APT 6
22        230 Clairmont Ave FLOOR 2
23            7708 Westmoreland Ave
24             2507 Wedgemere St #2
25                 252 Atwood St #1
26                    1315 Diana St
Name: Address1, dtype: objec